# Capstone Project: Office Launch Delivery Startup

## Introduction

### Background and Description of the Problem

The client is an  Office Supplier in Toronto which services  small to medium offices. They have saturated the market in their own area and are now looking to expand into a nearby city. The two options under consideration by the client are Toronto and Montreal. As this is will be the client’s first business expansion, it is important that out of the two possible locations, they choose the city offering the best possible outcome. 
 
During this process another aspect of the clients business should also be taken into consideration. This is the fact that potential customers expect delivery within the hour, so any business location would have to be close to a number of customer sites.

The data science problem can be described using the  following questions: 

Part One: Which out of two possible locations has the greater number of potential customers? 

Part Two: In the city chosen in answer to part one, which city borough would offer the greatest catchment area for the business?

## I. Data preparation and analysis: 
## Toronto

Importing required Libraries

In [1]:
import numpy as np
import types
import pandas as pd
import types

Toronto borough information was scraped from wikipedia as a .csv file which was then imported as a pandas dataframe

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
df1 = pd.read_csv("Toronto_boroughs.csv") #, sep='\t'
print(df1.shape)
print(df1.head())
#print(df1)

(13, 5)
  PostCode           Borough      Neighborhood   Latitude  Longitude
0      M3A        North York         Parkwoods  43.753259 -79.329656
1      M5A  Downtown Toronto      Harbourfront  43.654260 -79.360636
2      M7A      Queen's Park      Queen's Park  43.662301 -79.389494
3      M9A         Etobicoke  Islington Avenue  43.667856 -79.532242
4      M1B       Scarborough             Rouge  43.806686 -79.194353


Geo data for Toronto Boroughs was imported as a .csv file.

In [5]:
# The code was removed by Watson Studio for sharing.

Using geopy library to get the latitude and longitude values of Toronto

In [6]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Creating a map of Toronto with neighbourhoods superimposed on top.

In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

toronto = df1

for lat, lng, borough in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Using the foursquare API a search was made for potential clients using foursquare's office category. 
We will be searching within a 750 meters radius of the centre of Toronto.

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
LIMIT = 100
radius = 750

CLIENT_ID="TAAXKQEFKEXXB0S0NBTOKZQKTWEZQLC2WOE0C0RDOMK1TT0W"
CLIENT_SECRET="F2P35ORT1JVWD5WETI2R1YXDCD5303HTJZSYDQBHPGGBTAFD"
VERSION="20180602"

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d124941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
#results
results['response']['groups'][0]['items'][0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ae5df5af964a520c4a221e3',
  'name': 'Bell Trinity Square',
  'location': {'address': '483 Bay St.',
   'crossStreet': 'at Albert St.',
   'lat': 43.65347479872822,
   'lng': -79.38246987630343,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65347479872822,
     'lng': -79.38246987630343}],
   'distance': 385,
   'postalCode': 'M5G 2C9',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['483 Bay St. (at Albert St.)',
    'Toronto ON M5G 2C9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d124941735',
    'name': 'Office',
    'pluralName': 'Offices',
    'shortName': 'Office',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referral

The resulting .json file was normalized and imported into a pandas dataframe.

In [10]:
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

In [11]:
offices = results['response']['groups'][0]['items']
    
nearby_offices_toronto = json_normalize(offices) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_offices_toronto = nearby_offices_toronto.loc[:, filtered_columns]
nearby_offices_toronto.head()
nearby_offices_toronto['venue.categories'] = nearby_offices_toronto.apply(get_category_type, axis=1)

nearby_offices_toronto.columns = [col.split(".")[-1] for col in nearby_offices_toronto.columns]

nearby_offices_toronto.head()

,name,categories,lat,lng
0,Bell Trinity Square,Office,43.653475,-79.382470
1,Bay Adelaide Centre,Office,43.650879,-79.380003
2,Kids Help Phone,Office,43.654699,-79.387089
3,Tangerine Café,Bank,43.653937,-79.379722
4,Atrium On Bay,Office,43.656501,-79.381900


In [12]:
nearby_offices_toronto.shape

(97, 4)

The resulting data frame show there are 97 potential customers within a five mile radius of the center of Toronto city.

### Montreal

The same process was then repeated with data for the city of Montreal

Montreal borough information was scraped from wikipedia as a .csv file which was then imported as a pandas dataframe

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
df2 = pd.read_csv("Montreal_boroughs.csv", sep='\t')
df2.head()

,Postal code,Borough,Latitude,Longitude
0,H3M,Ahuntsic-Cartierville,45.53856,-73.69266
1,H1K,Anjou,45.60933,-73.54508
2,H3W,Côte-des-Neiges–Notre-Dame-de-Grâce,45.49069,-73.63321
3,H8S,Lachine,45.43665,-73.68510
4,H8N,Lasalle,45.43890,-73.62583


In [15]:
df2.shape

(19, 4)

Geo data for Montreal Boroughs was imported as a .csv file.

In [16]:
# The code was removed by Watson Studio for sharing.

Using geopy library to get the latitude and longitude values of Montreal

In [17]:
address = 'Montreal, Quebec, Canada'

geolocator = Nominatim(user_agent="montreal_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Montreal are 45.4972159, -73.6103642.


Creating a map of Montreal with neighbourhoods superimposed on top.

In [18]:
map_montreal = folium.Map(location=[latitude, longitude], zoom_start=10)
montreal = df2

for lat, lng, borough in zip(montreal['Latitude'], montreal['Longitude'], montreal['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal

Using the foursquare API a search was made for potential clients using foursquare's office category. This was set within a five mile radius of the centre of Montreal.

In [19]:
LIMIT = 100

radius = 750

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d124941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results['response']['groups'][0]['items'][0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '58ac237fea29b818aba0cbf8',
  'name': 'S&P Transport',
  'location': {'address': '6374 Avenue Papineau',
   'lat': 45.49681188392745,
   'lng': -73.60927820205688,
   'labeledLatLngs': [{'label': 'display',
     'lat': 45.49681188392745,
     'lng': -73.60927820205688}],
   'distance': 95,
   'postalCode': 'H2G 2W9',
   'cc': 'CA',
   'city': 'Montréal',
   'state': 'QC',
   'country': 'Canada',
   'formattedAddress': ['6374 Avenue Papineau',
    'Montréal QC H2G 2W9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d124941735',
    'name': 'Office',
    'pluralName': 'Offices',
    'shortName': 'Office',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-58ac237fea29b818aba0cbf8-0'}

The resulting .json file was normalized and imported into a pandas dataframe.

In [20]:
offices = results['response']['groups'][0]['items']
    
nearby_offices_montreal = json_normalize(offices) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_offices_montreal =nearby_offices_montreal.loc[:, filtered_columns]

nearby_offices_montreal['venue.categories'] = nearby_offices_montreal.apply(get_category_type, axis=1)

nearby_offices_montreal.columns = [col.split(".")[-1] for col in nearby_offices_montreal.columns]

nearby_offices_montreal.tail()

,name,categories,lat,lng
8,Kronos Canadian Systems,Office,45.495478,-73.618936
9,Virtuo360 Virtual Tour,Advertising Agency,45.496909,-73.608657
10,Montreally,Advertising Agency,45.496304,-73.608806
11,MontréAgence,Advertising Agency,45.497300,-73.610158
12,et voila canada,Advertising Agency,45.497774,-73.611016


In [21]:
nearby_offices_montreal.shape

(13, 4)

The resulting data frame show there are 100 potential customers within a five mile radius of the center of Montreal city.

### Results of Problem/Question 1

In [22]:
print('{} Nearby offices in Montreal.'.format(nearby_offices_montreal.shape[0]))
print('{} Nearby offices in Toronto.'.format(nearby_offices_toronto.shape[0]))

13 Nearby offices in Montreal.
97 Nearby offices in Toronto.


### Conclusion of Problem/Question 1

The city that should be analysed further is Toronto as it offers the greater number of offices located in proximity.

## Problem/Question 2: 

In the city chosen in answer to part one, which city borough would offer the best place location delivery parking facility?

### Data Preparation and Analysis

In [23]:
#selected_df=df1
selected_df=toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d124941735'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_offices = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_offices.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_offices)

In [25]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [26]:
toronto_offices = getNearbyVenues(names=selected_df['Borough'],
                                   latitudes=selected_df['Latitude'],
                                   longitudes=selected_df['Longitude']
                                    )

North York
Downtown Toronto
Queen's Park
Etobicoke
Scarborough
East York
York
East Toronto
North York
West Toronto
North York
Central Toronto
Mississauga


In [27]:
print (toronto_offices.shape)
toronto_offices.head()

(72, 7)


,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,CityView Roofing System Ltd,43.752265,-79.327941,Office
1,North York,43.753259,-79.329656,Valleywood Corporate Centre,43.754036,-79.332662,Office
2,North York,43.753259,-79.329656,Emergency Plumbing Toronto Inc.,43.754454,-79.333319,Office
3,North York,43.753259,-79.329656,Lucid Blue Consulting,43.756407,-79.325866,Office
4,North York,43.753259,-79.329656,D2 Designs,43.754568,-79.332035,Coworking Space


In [28]:
toronto_offices.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,1,1,1,1,1,1
Downtown Toronto,23,23,23,23,23,23
East Toronto,1,1,1,1,1,1
East York,4,4,4,4,4,4
Mississauga,8,8,8,8,8,8
North York,5,5,5,5,5,5
Queen's Park,22,22,22,22,22,22
Scarborough,4,4,4,4,4,4
York,4,4,4,4,4,4


Analyze each Borough of Toronto

Segmenting and clustering boroughs in Toronto by slicing the original and creating a new dataframe for Toronto borough data.

In [29]:
toronto_onehot = pd.get_dummies(toronto_offices[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Borough'] = toronto_offices['Borough'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Advertising Agency,Building,Coworking Space,Government Building,Medical Center,Office,Tech Startup
0,North York,0,0,0,0,0,1,0
1,North York,0,0,0,0,0,1,0
2,North York,0,0,0,0,0,1,0
3,North York,0,0,0,0,0,1,0
4,North York,0,0,1,0,0,0,0


In [30]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Advertising Agency,Building,Coworking Space,Government Building,Medical Center,Office,Tech Startup
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
1,Downtown Toronto,0.043478,0.043478,0.043478,0.000000,0.000000,0.826087,0.043478
2,East Toronto,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
4,Mississauga,0.000000,0.125000,0.125000,0.000000,0.000000,0.750000,0.000000
5,North York,0.000000,0.000000,0.200000,0.000000,0.000000,0.800000,0.000000
6,Queen's Park,0.000000,0.000000,0.090909,0.045455,0.045455,0.727273,0.090909
7,Scarborough,0.000000,0.000000,0.250000,0.000000,0.000000,0.750000,0.000000
8,York,0.250000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000


In [31]:
toronto_grouped.shape

(9, 8)

Print each Borough with its 5 most common venues.

In [32]:
num_top_offices = 5

for Borough in toronto_grouped['Borough']:
    print("----"+Borough+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == Borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_offices))
    print('\n')

----Central Toronto----
                 venue  freq
0               Office   1.0
1   Advertising Agency   0.0
2             Building   0.0
3      Coworking Space   0.0
4  Government Building   0.0


----Downtown Toronto----
                venue  freq
0              Office  0.83
1  Advertising Agency  0.04
2            Building  0.04
3     Coworking Space  0.04
4        Tech Startup  0.04


----East Toronto----
                 venue  freq
0   Advertising Agency   1.0
1             Building   0.0
2      Coworking Space   0.0
3  Government Building   0.0
4       Medical Center   0.0


----East York----
                 venue  freq
0               Office   1.0
1   Advertising Agency   0.0
2             Building   0.0
3      Coworking Space   0.0
4  Government Building   0.0


----Mississauga----
                 venue  freq
0               Office  0.75
1             Building  0.12
2      Coworking Space  0.12
3   Advertising Agency  0.00
4  Government Building  0.00


----North York----

##### Place this into a pandas dataframe in descending order, display the top 3 venues for each neighbourhood.

In [33]:
def return_most_common_venues(row, num_top_offices):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_offices]

In [34]:
num_top_offices = 3

indicators = ['st', 'nd', 'rd']

columns = ['Borough']
for ind in np.arange(num_top_offices):
    try:
        columns.append('{}{} Most Common Office'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Office'.format(ind+1))

boroughs_offices_sorted = pd.DataFrame(columns=columns)
boroughs_offices_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_offices_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_offices)

boroughs_offices_sorted.head()

,Borough,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
0,Central Toronto,Office,Tech Startup,Medical Center
1,Downtown Toronto,Office,Tech Startup,Coworking Space
2,East Toronto,Advertising Agency,Tech Startup,Office
3,East York,Office,Tech Startup,Medical Center
4,Mississauga,Office,Coworking Space,Building


Now running k-means to cluster the neighbourhoods into 5 clusters. Then create a new dataframe to include the cluster.

In [35]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 2, 1, 3, 2, 0, 2, 0, 4], dtype=int32)

In [36]:
boroughs_offices_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_offices

toronto_merged = toronto_merged.join(boroughs_offices_sorted.set_index('Borough'), on='Borough',how='right')

toronto_merged.head()

,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
0,North York,43.753259,-79.329656,CityView Roofing System Ltd,43.752265,-79.327941,Office,0,Office,Coworking Space,Tech Startup
1,North York,43.753259,-79.329656,Valleywood Corporate Centre,43.754036,-79.332662,Office,0,Office,Coworking Space,Tech Startup
2,North York,43.753259,-79.329656,Emergency Plumbing Toronto Inc.,43.754454,-79.333319,Office,0,Office,Coworking Space,Tech Startup
3,North York,43.753259,-79.329656,Lucid Blue Consulting,43.756407,-79.325866,Office,0,Office,Coworking Space,Tech Startup
4,North York,43.753259,-79.329656,D2 Designs,43.754568,-79.332035,Coworking Space,0,Office,Coworking Space,Tech Startup


In [37]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters and determine discriminating potential customer categories for each cluster - assigning a name to each cluster

#### Cluster 1: Office/Coworking Space

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
0,43.753259,-79.327941,Office,0,Office,Coworking Space,Tech Startup
1,43.753259,-79.332662,Office,0,Office,Coworking Space,Tech Startup
2,43.753259,-79.333319,Office,0,Office,Coworking Space,Tech Startup
3,43.753259,-79.325866,Office,0,Office,Coworking Space,Tech Startup
4,43.753259,-79.332035,Coworking Space,0,Office,Coworking Space,Tech Startup
50,43.806686,-79.195614,Office,0,Office,Coworking Space,Tech Startup
51,43.806686,-79.189754,Office,0,Office,Coworking Space,Tech Startup
52,43.806686,-79.198802,Office,0,Office,Coworking Space,Tech Startup
53,43.806686,-79.199744,Coworking Space,0,Office,Coworking Space,Tech Startup


#### Cluster 2: Advertising Agency / Tech Startup

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
62,43.676357,-79.287665,Advertising Agency,1,Advertising Agency,Tech Startup,Office


#### Cluster 3: Office / Tech Startup

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
5,43.654260,-79.361270,Office,2,Office,Tech Startup,Coworking Space
6,43.654260,-79.363374,Office,2,Office,Tech Startup,Coworking Space
7,43.654260,-79.358830,Office,2,Office,Tech Startup,Coworking Space
8,43.654260,-79.358908,Office,2,Office,Tech Startup,Coworking Space
9,43.654260,-79.364516,Office,2,Office,Tech Startup,Coworking Space
10,43.654260,-79.357099,Office,2,Office,Tech Startup,Coworking Space
11,43.654260,-79.364923,Office,2,Office,Tech Startup,Coworking Space
12,43.654260,-79.365648,Office,2,Office,Tech Startup,Coworking Space
13,43.654260,-79.357581,Office,2,Office,Tech Startup,Coworking Space
14,43.654260,-79.364591,Office,2,Office,Tech Startup,Coworking Space


#### Cluster 4: Office / Tech Startup

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
54,43.706397,-79.313175,Office,3,Office,Tech Startup,Medical Center
55,43.706397,-79.312749,Office,3,Office,Tech Startup,Medical Center
56,43.706397,-79.314127,Office,3,Office,Tech Startup,Medical Center
57,43.706397,-79.308729,Office,3,Office,Tech Startup,Medical Center
63,43.728020,-79.392012,Office,3,Office,Tech Startup,Medical Center


#### Cluster 5: Office / Advertising Agency

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Office,2nd Most Common Office,3rd Most Common Office
58,43.693781,-79.423289,Office,4,Office,Advertising Agency,Tech Startup
59,43.693781,-79.426818,Office,4,Office,Advertising Agency,Tech Startup
60,43.693781,-79.422848,Office,4,Office,Advertising Agency,Tech Startup
61,43.693781,-79.425101,Advertising Agency,4,Office,Advertising Agency,Tech Startup


### Results of Problem/Question 2

Cluster 3 contains the greatest number of potential offices/customers. Potential customers are in the Offices/ Tech Startup Offices. 

Cluster 3 consists of three boroughs which are Downtown Toronto, Queen's Park and Mississauga.

### Conclusion of Problem/Question 2

The city boroughs which would offer the greatest place for the business are 
Downtown Toronto and Queen's Park. The third option while having a large concentration of potential customers is isolated and would reduce opportunities for future business growth. Between first two options Downtown Toronto would be preferable having more potential offices/customers. Our winer is Downtown Toronto

# Conclusion of Data Analysis

After data collection, cleaning and analysis we can answer the initial data science problem posed in the introduction of this project.

Problem/Question 1: Which out of two possible locations (Toronto and Montreal) has the greater number of potential offices/customers?  
Answer: Toronto as it offers the greater number of potential offices/customers.

Problem/Question 2: In the city chosen in answer to Problem/Question 1 (in our case Toronto), which city borough would offer the best place for the business?  
Answer: The city borough of Toronto which would offer the best place for the business is Downtown Toronto.

